In [7]:
import pickle
import cv2
import os
import sys
import numpy as np 
import pandas as pd 
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import glob
from PIL import Image

In [8]:
model = pickle.load(open('mainmodel.pkl','rb'))

In [9]:
import time

In [10]:
waste_classes = {
    '0' : 'Book/Magazine',
    '1' : 'Cardboard',
    '2' : 'Cutlery/Utensils',
    '3' : 'Electrical Device',
    '4' : 'Glass Bottle',
    '5' : 'Glass Jar',
    '6' : 'Metal Cans',
    '7' : 'Newspaper',
    '8' : 'Paper',
    '9' : 'Plastic Bag',
    '10' : 'Plastic Bottle',
    '11' : 'Plate/ Tray',
    '12' : 'Toiletry',
    '13' : 'Vegetable/Fruit'
} 

In [11]:
read = lambda imname: np.asarray(Image.open(imname).convert("RGB").resize((224,224)))

In [6]:
video = cv2.VideoCapture(0)
final_result = ''
while True:
    
        _, frame = video.read()
 
        start_point = (160,160)
        end_point = (440,440)
        color =  (0,225,0)
        thickness = 2
        
        frame = cv2.rectangle(frame, start_point , end_point , color , thickness) 
        tic1 = time.time()
        #Resizing into 224x224 because we trained the model with this image size and changing to RGB channel
        im = Image.fromarray(frame, 'RGB') 
        im = im.resize((224,224))
        img_array = np.array(im)

        #Our keras model used a 4D tensor, (images x height x width x channel)
        #So changing dimension 224x224x3 into 1x224x224x3 
        img_array = np.expand_dims(img_array, axis=0)

        #Calling the predict method on model
        prediction = model.predict(img_array)
        class_prediction = np.argmax(prediction, axis=1)
    
        answer = [v for k,v in waste_classes.items() if str(class_prediction[0]) == k][0] 
        key=cv2.waitKey(3)
        if key == ord('t'):
            cv2.putText(frame, answer, (50,50) , cv2.FONT_HERSHEY_SIMPLEX , 2 ,(255, 0, 0),
                2,  
                cv2.LINE_4)
            key=cv2.waitKey(5)
            
            
            
        toc1 = time.time()
        # Press s only if you want to get DIY ideas 
        if key == ord('s'):
            tic2 = time.time()
            cv2.imwrite(filename='saved_img.jpg', img=frame)
            img_new = cv2.imread('saved_img.jpg', cv2.IMREAD_GRAYSCALE)
            img_new = cv2.imshow("Captured Image", img_new)
            
            #Here we are storing the image whose DIY hacks we want to know in the testing folder, in a processed form
            print("Processing image...")
            img_ = cv2.imread('saved_img.jpg', cv2.IMREAD_ANYCOLOR)
            print("Resizing image to 224x224 scale...")
            img_ = cv2.resize(img_,(224,224))
            print("Resized...")
            img_resized = cv2.imwrite(filename=r'C:\Users\NDH60042\MAJOR PROJECT\test\saved_img-final.jpg', img=img_)
            print("Image saved!")
            
            #Here we are reading the image that just got stored in the test folder
            test_data = []
            testpath_ = r'C:\Users\NDH60042\MAJOR PROJECT\test'
            testdata_path = os.path.join(testpath_ ,'*g')
            test_files = glob.glob(testdata_path)
            for f in test_files:
                ims = [read(f)]
                test_data += ims
      
            test_array = np.array(test_data , dtype = 'uint8')
            
            #Predicting the class
            prediction = model.predict(test_array)
            class_prediction = np.argmax(prediction, axis = 1)
            
            answer = [v for k,v in waste_classes.items() if str(class_prediction[0]) == k][0]     
            
            cv2.putText(img_new, answer, (50,50) , cv2.FONT_HERSHEY_SIMPLEX , 2 ,(255, 0, 0),
                        2, cv2.LINE_4)
            
            toc2 = time.time()
            #Storing the result whose DIY hacks we want
            final_result = answer
            cv2.waitKey(1650)
            cv2.destroyAllWindows()
            
            break

        cv2.imshow("video", frame)
       
        if key == ord('q'):
                break
video.release()
cv2.destroyAllWindows()



In [16]:
import webbrowser

In [31]:
if len(final_result.split())==2:
    search = "https://www.google.com/search?q=DIY+Reusing+Ideas+for"+final_result.split()[0]+final_result.split()[1]
else:
    search = "https://www.google.com/search?q=DIY+Reusing+Ideas+for"+final_result

In [32]:
webbrowser.open(search)

True

<h1>FLASK FUNCTION </h1>

In [26]:
image_file = r'C:\Users\NDH60042\MAJOR PROJECT\paper1.jpg'

In [28]:
from keras.applications.resnet50 import preprocess_input,decode_predictions

In [46]:
def get_prediction(image_file):
    image = tf.keras.preprocessing.image.load_img(
        image_file, target_size=(224,224)
    )
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.array(image)
    image = np.expand_dims(image, axis = 0)
    image = preprocess_input(image)
    
    prediction = model.predict(image)
    class_prediction = np.argmax(prediction)
    print(prediction)
    waste_classes = {
    '0' : 'Book/Magazine',
    '1' : 'Cardboard',
    '2' : 'Cutlery/Utensils',
    '3' : 'Electrical Device',
    '4' : 'Glass Bottle',
    '5' : 'Glass Jar',
    '6' : 'Metal Cans',
    '7' : 'Newspaper',
    '8' : 'Paper',
    '9' : 'Plastic Bag',
    '10' : 'Plastic Bottle',
    '11' : 'Plate/ Tray',
    '12' : 'Toiletry',
    '13' : 'Vegetable/Fruit'
     }
    result = [v for k,v in waste_classes.items() if k== str(class_prediction)][0]
    accuracy = prediction[0][class_prediction]*100
    return result,accuracy

In [47]:
result,accuracy = get_prediction(image_file)

[[1.91285342e-01 2.20723942e-01 8.53287759e-07 1.44125399e-04
  2.72564157e-05 3.95585585e-06 2.33550952e-03 3.05173267e-03
  5.81583381e-01 1.28065716e-04 1.37375382e-05 6.90758927e-04
  1.13082542e-05 1.01024895e-07]]


In [44]:
result

'Paper'

In [45]:
accuracy

58.15833806991577

In [48]:
print(tf.__version__)

2.1.0


In [50]:
import keras
print(keras.__version__)

2.3.1


In [5]:
str1 = 'Glass Jar'
str1.split()[1]

'Jar'